In [12]:
import pandas as pd
import numpy as np
from PyHa.statistics import *
from PyHa.IsoAutio import *

C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\17149\miniconda3\envs\species-id\lib\site-pack

In [2]:
full_labels_df = pd.read_csv("MDD_Xeno_Canto_DSC180_Labels.csv")

In [3]:
full_labels_df

,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME
0,Aburria-aburri-257423.wav,24.6335,2.3614,3.3205,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,02:13:53
1,Aburria-aburri-257423.wav,24.6335,6.7320,2.0103,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,02:13:53
2,Aburria-aburri-257423.wav,24.6335,19.2438,3.4505,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,02:13:53
3,Aburria-aburri-387212.wav,84.4800,0.6011,3.0800,24000,0,44100,Aburria aburri Wattled Guan,184.388,alarce,11/22/2021,03:28:01
4,Aburria-aburri-387212.wav,84.4800,77.6811,3.1400,24000,0,44100,Aburria aburri Wattled Guan,184.385,alarce,11/22/2021,03:28:01
...,...,...,...,...,...,...,...,...,...,...,...,...
31360,Zonotrichia-capensis-232609.wav,98.8560,63.2772,1.8301,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.174,JacobGlennAyers,10/10/2021,02:06:40
31361,Zonotrichia-capensis-232609.wav,98.8560,34.1554,2.4702,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.135,JacobGlennAyers,10/10/2021,02:06:40
31362,Zonotrichia-capensis-377483.wav,18.2596,0.5762,3.2199,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.527,JacobGlennAyers,10/18/2021,17:33:36
31363,Zonotrichia-capensis-377483.wav,18.2596,6.8160,2.1500,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.543,JacobGlennAyers,10/18/2021,17:33:36


In [5]:
clips = full_labels_df["IN FILE"].unique()

'Aburria-aburri-257423.wav'

In [ ]:
# I think I can build what I need by using PyHa functions. I need to combine the waveform logic that I worked out 
# for the statistics and combine it with the chunker logic that I have worked out.

# initializing an empty array 



In [20]:
# specifiying a uniform length for the annotations in seconds
annotation_len = 3.0
# initializing the output dataframe
output_dict = {
    "IN FILE"     : [],
    "CLIP LENGTH" : [],
    "OFFSET"      : [],
    "DURATION"    : [],
    "SAMPLE RATE" : [],
    "MANUAL ID"   : [],
    
}
# going through each
for clip in clips:
    clip_df = full_labels_df[full_labels_df["IN FILE"] == clip]
    birds = clip_df["MANUAL ID"].unique()
    sr = clip_df["SAMPLE RATE"].unique()[0]
    clip_len = clip_df["CLIP LENGTH"].unique()[0]
    #print(clip_len)
    # quick data sanitization to remove very short clips
    if clip_len < annotation_len:
        continue
    potential_annotation_count = int(clip_len)//int(annotation_len)
    #print(potential_annotation_count)
    # going through each species that was ID'ed in the clip
    #print("Clip Length:",clip_len)
    arr_len = int(clip_len*1000)
    for bird in birds:
        species_df = clip_df[clip_df["MANUAL ID"] == bird]
        human_arr = np.zeros((arr_len))
        # looping through each annotation
        for annotation in species_df.index:
            minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
            # Determining the end of a human label
            maxval = int(
                round(
                    (species_df["OFFSET"][annotation] +
                     species_df["DURATION"][annotation]) *
                    1000,
                    0))
            # Placing the label relative to the clip
            human_arr[minval:maxval] = 1
        # performing the chunk isolation technique on the human array
        #print("Array_Length:",human_arr.shape[0])
        for ndx in range(potential_annotation_count):
            chunk_start = ndx * (annotation_len*1000)
            #print("Chunk Start:",chunk_start)
            chunk_end = min((ndx+1)*annotation_len*1000,arr_len)
            chunk = human_arr[int(chunk_start):int(chunk_end)]
            if max(chunk) >= 0.5:
                annotation_start = chunk_start / 1000
                #annotation_end = chunk_end / 1000
                #updating the dictionary
                output_dict["IN FILE"].append(clip)
                output_dict["CLIP LENGTH"].append(clip_len)
                output_dict["OFFSET"].append(annotation_start)
                output_dict["DURATION"].append(annotation_len)
                output_dict["SAMPLE RATE"].append(sr)
                output_dict["MANUAL ID"].append(bird)

In [22]:
# creating a dataframe
output_df = pd.DataFrame.from_dict(output_dict)

In [26]:
output_df.to_csv("MDD_Xeno_Canto_DSC180_Labels_uniform_3s.csv",index=False)